# NumWorks experimentation 16 November
It seems clear that the optimal number of workers for my system (4 cores 4 threads) is 2 or 0. This is posisbly due to the fact that using `num_workers=4` results in far to much context switching, whilst setting `num_workers=0` means that only the main thread is operating. This is effective in that the CPU is constantly active on the DataLoading task.

The Total CPU time for `num_workers=2` is notably faster than `num_workers=4` and much the same as `num_workers=3`. This seems to strike a good balance between parallelism and minimal context switching we suspect.

In terms of Wall time however, both `num_workers=2` and `num_workers=0` have the similar execution time but 0 is overall better.

![Charts](DataLoader_num_workers_chart.png)

| Batch size | User(4) | Sys(4) | Total(4) | wall(4) | User(2) | Sys(2) | Total(2) | Wall(2) | User(3) | Sys(3) | Total(3) | Wall(3) | Wall(8) | Total(0) | Wall(0) |
|------------|---------|--------|----------|---------|---------|--------|----------|---------|---------|--------|----------|---------|---------|----------|---------|
| 1          | 1,22    | 1,11   | 3,29     | 0,945   | 2,58    | 0      | 2,58     | 0,089   | 1,85    | 0,082  | 1,93     | 0,262   | 0,5     | 691      | 0,691   |
| 5          | 2,74    | 10,9   | 13,7     | 8,26    | 5,84    | 1,57   | 7,41     | 3,53    | 10,7    | 0,3    | 11       | 4,25    | 8,13    | 3040     | 3,05    |
| 10         | 12      | 10,4   | 22,4     | 14,4    | 5,41    | 0,396  | 5,81     | 3,93    | 8,69    | 0,2    | 8,9      | 6,52    | 22,4    | 7980     | 7,47    |
| 15         | 12,7    | 9,18   | 21,3     | 25,2    | 7,56    | 14,3   | 21,8     | 17,1    | 7,77    | 16,2   | 24       | 15,4    | 16,5    | 12100    | 11,6    |
| 20         | 19,2    | 26,2   | 45,4     | 33,3    | 9,87    | 8,58   | 18,4     | 14,4    | 12      | 4      | 16,1     | 13,1    | 54      | 13500    | 13,1    |
| 30         | 7,61    | 26,4   | 34       | 30,8    | 30,4    | 4,55   | 35       | 27,5    | 18,6    | 15,5   | 34,1     | 24,9    |         | 24400    | 24      |
| 60         | 42,9    | 29,6   | 72,5     | 71      | 21,5    | 32,7   | 54,3     | 56,9    | 43,3    | 11,3   | 54,6     | 66      |         | 44800    | 44,4    |
| 120        | 80,6    | 48,8   | 129      | 166     | 26,5    | 47,9   | 74,4     | 87      | 95      | 67,7   | 163      | 207     |         | 89000    | 89      |
| 256        | 256     | 293    | 551      | 720     | 108     | 114    | 222      | 359     |         |        |          |         |         | 225000   | 225     |

In [1]:
from data_utils.S3DISDataLoader import S3DISDataset
import data_utils.MastersDataLoader as MastersDataLoader
import numpy as np
import torch
import time

from types import SimpleNamespace

root = 'data/s3dis/songomnara'
NUM_POINT = 2048
NUM_CLASSES = 2
BATCH_SIZE = 10
args = {'test_area': 2, 'block_size': 1,
        'test_sample_rate': 1, 'shuffle_training_data': True}
args = SimpleNamespace(**args)


In [2]:
print("start loading training data ...")
TRAIN_DATASET = S3DISDataset(split='train', data_root=root, num_point=NUM_POINT, test_area=args.test_area,
                             block_size=args.block_size, sample_rate=1.0, transform=None, num_classes=NUM_CLASSES)


  0%|          | 0/8 [00:00<?, ?it/s]

start loading training data ...


100%|██████████| 2/2 [00:00<00:00, 171.26it/s]

Labelweights=[1.        1.3157682]
Totally 9572 samples in train set.
start loading test data ...
Labelweights=[1.        1.0052938]
Totally 37 samples in test set.


In [3]:
test_points = 0
for room in TEST_DATASET.room_points:
    test_points += room.shape[0]

train_points = 0
for room in TRAIN_DATASET.room_points:
    train_points += room.shape[0]

test_points/train_points*100


0.39546183972797966

In [3]:
# 4: default, 
# 0: single threaded and wall time=user time
# 2:
for num_workers in [0,2]: 
    for batach_size in [1,1, 256]:
        print(f"\n num_workers = {num_workers}, batch_size = {batach_size}")
        trainDataLoader = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=batach_size,
                                                      shuffle=args.shuffle_training_data, num_workers=num_workers,
                                                      pin_memory=True, drop_last=True,
                                                      worker_init_fn=lambda x: np.random.seed(x + int(time.time())))
        train_iter = iter(trainDataLoader)
        %time points, target, room_idx = next(train_iter)



 num_workers = 0, batch_size = 1
CPU times: user 1.68 s, sys: 392 ms, total: 2.07 s
Wall time: 2.09 s

 num_workers = 0, batch_size = 1
CPU times: user 17.7 ms, sys: 0 ns, total: 17.7 ms
Wall time: 17.7 ms

 num_workers = 0, batch_size = 256
DEBUG: increasing block size to 2
DEBUG: increasing block size to 4
DEBUG: increasing block size to 8
CPU times: user 3min 35s, sys: 10 s, total: 3min 45s
Wall time: 3min 46s

 num_workers = 2, batch_size = 1
CPU times: user 130 ms, sys: 127 ms, total: 257 ms
Wall time: 2.13 s

 num_workers = 2, batch_size = 1
CPU times: user 2.02 ms, sys: 101 µs, total: 2.12 ms
Wall time: 20.3 ms

 num_workers = 2, batch_size = 256
DEBUG: increasing block size to 16
CPU times: user 122 ms, sys: 114 ms, total: 235 ms
Wall time: 7min 9s
